In [6]:
from datetime import datetime
import requests
from markdown import Markdown
import pandas as pd

In [7]:
n_condos = requests.get(
    """
    https://phl.carto.com/api/v2/sql?q=SELECT count(*) FROM condominium
    """
).json()["rows"][0]["count"]

print(f"There are {n_condos} condos as of {datetime.now().isoformat()}")

There are 42883 condos as of 2021-01-10T23:37:02.569174


In [8]:
condo_results = requests.get(
    """
https://phl.carto.com/api/v2/sql?q=SELECT * FROM condominium limit 1
"""
).json()["rows"]
condo_results

[{'cartodb_id': 1,
  'the_geom': None,
  'the_geom_webmercator': None,
  'objectid': 181,
  'basereg': '005N220319',
  'condoparcel': '0319',
  'condounit': '209',
  'condo_name': 'CARRIAGE HOUSE CONDOMINIUM',
  'inactdate': None,
  'mapref': '005N220132',
  'orig_date': '1986-01-13T00:00:00Z',
  'parcel': '0132',
  'recmap': '005N22',
  'status': 1}]

In [9]:
condo_names = requests.get(
    """
https://phl.carto.com/api/v2/sql?q=SELECT condo_name, mapref, count(*) FROM condominium group by condo_name, mapref
"""
).json()["rows"]
condo_names_df = pd.DataFrame(condo_names).sort_values("count", ascending=False)
condo_names_df

,condo_name,mapref,count
5024,THE PHILADELPHIAN,006N040072,794
391,CITYVIEW,004N220070,738
1936,SOCIETY HILL TOWERS,002S050253,703
2177,ACADEMY HOUSE,002S170097,607
914,2601 PARKWAY,008N020026,590
...,...,...,...
2999,,002S100148,1
3000,NAVAL SQUARE - 302 CAPTAIN'S WAY,006S090012,1
3002,THE VILLAS AT PACKER PARK -- 2116 VERONA DRIVE,033S080331,1
706,832 N 16TH ST,007N100200,1


There is no opa_account_number in this dataset, but there are two ways that this can be joined to one.

1) via the RTT dataset, by matching the rtt.matched_regmap to the condominiums.mapref (used below). There may be 
multiple records in the RTT dataset, but they *should* all point to the same opa_account_num.

2) via the [department of records](https://www.opendataphilly.org/dataset/property-parcels) and the opa_properties_public,
by matching the opa_properties_public.location to the department_of_records.addr_source (where status=1), and
then by matching the department_of_records.mapreg = condominiums.mapref

In [10]:
# via the RTT dataset, matching on condounit and cond.mapref that matches 2101 Market Street
condos_results = requests.get(
    """
    https://phl.carto.com/api/v2/sql?q=
    SELECT distinct(rtt.opa_account_num), cond.* FROM condominium cond join rtt_summary rtt on cond.mapref=rtt.matched_regmap
    where cond.mapref = '001N120451'
    and rtt.unit_num = cond.condounit
    """
).json()["rows"]

df = pd.DataFrame(condos_results)
df

,opa_account_num,cartodb_id,the_geom,the_geom_webmercator,objectid,basereg,condoparcel,condounit,condo_name,inactdate,mapref,orig_date,parcel,recmap,status
0,888094100,30681,None,None,32803,001N120452,0452,202,MURANO CONDOMINIUM,None,001N120451,None,0451,001N12,1
1,888094102,22622,None,None,29071,001N120453,0453,203,MURANO CONDOMINIUM,None,001N120451,None,0451,001N12,1
2,888094104,33905,None,None,29283,001N120454,0454,204,MURANO CONDOMINIUM,None,001N120451,None,0451,001N12,1
3,888094106,34763,None,None,31056,001N120455,0455,205,MURANO CONDOMINIUM,None,001N120451,None,0451,001N12,1
4,888094108,23348,None,None,29260,001N120456,0456,206,MURANO CONDOMINIUM,None,001N120451,None,0451,001N12,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
300,888094700,34569,None,None,29448,001N120752,0752,PH03,MURANO CONDOMINIUM,None,001N120451,None,0451,001N12,1
301,888094702,36141,None,None,29875,001N120753,0753,PH04,MURANO CONDOMINIUM,None,001N120451,None,0451,001N12,1
302,None,33899,None,None,30861,001N120643,0643,2808,MURANO CONDOMINIUM,None,001N120451,None,0451,001N12,2
303,None,34613,None,None,29492,001N120676,0676,3301,MURANO CONDOMINIUM,None,001N120451,None,0451,001N12,2


In [11]:
# via the RTT dataset, matching on condounit and rtt.opa_account_num for a unit at 2101 Market St
condos_results = requests.get(
    """
    https://phl.carto.com/api/v2/sql?q=
    SELECT distinct(rtt.opa_account_num), cond.* FROM condominium cond 
    left join rtt_summary rtt on cond.mapref=rtt.matched_regmap
    where rtt.opa_account_num = '888094148'
    and rtt.unit_num = cond.condounit
    """
).json()["rows"]

pd.DataFrame(condos_results)

,opa_account_num,cartodb_id,the_geom,the_geom_webmercator,objectid,basereg,condoparcel,condounit,condo_name,inactdate,mapref,orig_date,parcel,recmap,status
0,888094148,33782,None,None,31861,001N120476,0476,701,MURANO CONDOMINIUM,None,001N120451,None,0451,001N12,1


In [12]:
# via the RTT dataset, check if opa_account_num is condo
condos_results = requests.get(
    """
    https://phl.carto.com/api/v2/sql?q=
    SELECT distinct(rtt.opa_account_num), cond.* FROM condominium cond 
    left join rtt_summary rtt on cond.mapref=rtt.matched_regmap
    where rtt.opa_account_num = '888094148'
    and rtt.unit_num = cond.condounit
    """
).json()["rows"]

pd.DataFrame(condos_results)

,opa_account_num,cartodb_id,the_geom,the_geom_webmercator,objectid,basereg,condoparcel,condounit,condo_name,inactdate,mapref,orig_date,parcel,recmap,status
0,888094148,33782,None,None,31861,001N120476,0476,701,MURANO CONDOMINIUM,None,001N120451,None,0451,001N12,1
